In [1]:
import torch
import torch.nn as nn
import torchtext
from torchtext import data
from torchtext.vocab import Vectors
from torchtext.data import Iterator, BucketIterator
import spacy
import numpy as np


In [2]:
import jsonlines

In [6]:
with open('data/snli_1.0/snli_1.0_dev.jsonl', 'r+', encoding='utf-8') as f:
    for idx, item in enumerate(jsonlines.Reader(f)):
        print(item.keys())
        print(item.values())
        print(item)
        print((type(item)))
        if idx > 0:
            break

dict_keys(['annotator_labels', 'captionID', 'gold_label', 'pairID', 'sentence1', 'sentence1_binary_parse', 'sentence1_parse', 'sentence2', 'sentence2_binary_parse', 'sentence2_parse'])
dict_values([['neutral', 'entailment', 'neutral', 'neutral', 'neutral'], '4705552913.jpg#2', 'neutral', '4705552913.jpg#2r1n', 'Two women are embracing while holding to go packages.', '( ( Two women ) ( ( are ( embracing ( while ( holding ( to ( go packages ) ) ) ) ) ) . ) )', '(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP are) (VP (VBG embracing) (SBAR (IN while) (S (NP (VBG holding)) (VP (TO to) (VP (VB go) (NP (NNS packages)))))))) (. .)))', 'The sisters are hugging goodbye while holding to go packages after just eating lunch.', '( ( The sisters ) ( ( are ( ( hugging goodbye ) ( while ( holding ( to ( ( go packages ) ( after ( just ( eating lunch ) ) ) ) ) ) ) ) ) . ) )', '(ROOT (S (NP (DT The) (NNS sisters)) (VP (VBP are) (VP (VBG hugging) (NP (UH goodbye)) (PP (IN while) (S (VP (VBG holding) (S (VP (T